### 종합실습3
##### Requests 혹은 Selenium 모듈을 이용해 네이버 이미지 검색에서 아이유를 검색하고, 검색결과 이미지들을 PC에 저장해 보세요.

### 1. Requests로 크롤링하기

In [ ]:
import json
import urllib
import requests

#아래에 코드 작성
search_query = "아이유"
url = "https://s.search.naver.com/p/c/image/search.naver"
param = {
    "where": 'image',
    "rev": '44',
    "section": 'image',
    "query": search_query,
    "ac": '0',
    "aq": '0',
    "spq": '1',
    "nx_search_query": search_query,
    "res_fr": '0',
    "res_to": '0',
    "datetype": '0',
    "startdate": '0',
    "enddate": '0',
    "nso": 'so:r,a:all,p:all',
    "json_type": '6',
    "nlu_query": '{"r_category":"12 11"}',
    "nqx_theme": '{"theme":{"main":{"name":"people_star","os":"159229","pkid":"1"}}}',
    "gif": '0',
    "optStr":' ',
    "ccl": '0',
    "x_image":' ',
    "display": '100',
    "start": '1'
}
res = requests.get(url, params=param)
json_str = res.text[3:-4]
json_data = json.loads(json_str)

i = 0
for img in json_data["items"]:
    this_img_url = urllib.parse.unquote(img["originalUrl"]) #url encode 된 것을 decode
    this_image = requests.get(this_img_url, verify=False)
    this_image_type = this_image.headers["Content-Type"]
    if "jpeg" in this_image_type or "jpg" in this_image_type or "png" in this_image_type:
        this_ext = this_image_type.split("/")[1]
        i += 1
        print("{}번째 이미지 다운로드 중".format(i), end="\r")
    open("../outputs/images/naver/"+search_query+"_img"+str(i)+"."+this_ext, "wb").write(this_image.content)
print("{} 이미지 다운로드 완료".format(search_query), end="\r")

### 2. Selenium으로 크롤링하기

In [ ]:
import time
import urllib
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

#아래에 코드 작성
search_query = "아이유"
url = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query="+search_query
                   
driver = webdriver.Chrome("chromedriver.exe")
driver.get(url)

for i in range(1, 6):
    unit = 1000
    driver.execute_script("window.scrollTo(0, {})".format(i*unit))
    time.sleep(3)

html_text = driver.page_source
soup = BeautifulSoup(html_text, "html.parser")
images = soup.select(".thumb > a > img")
i = 0
for img in images:
    this_img_url = img.attrs["src"]
    #이미지 콘텐츠 다운로드는 requests 모듈을 써야 함
    if this_img_url[:4] == "http":
        this_image = requests.get(this_img_url)
        this_image_type = this_image.headers["Content-Type"]
        if "jpeg" in this_image_type or "jpg" in this_image_type or "png" in this_image_type:
            this_ext = this_image_type.split("/")[1]
            i += 1
            print("{}번째 이미지 다운로드 중".format(i), end="\r")
            open("../outputs/images/naver/"+search_query+"_img"+str(i)+"."+this_ext, "wb").write(this_image.content)
print("{} 이미지 다운로드 완료".format(search_query))